In [1]:
from __future__ import print_function
import numpy as np
import torch

import time
import random
import math
import networkx as nx
# import node2vec #随机游走相关
from utils import *
import os
# from config2 import *
# from evaluation import *
from tkinter import _flatten
import classify
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import scipy.io as sio
from sklearn.metrics import *
from sklearn.metrics.pairwise import cosine_similarity
import scipy.io as sio
from sklearn.cluster import KMeans
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn import metrics
from munkres import Munkres, print_matrix
from collections import Counter

In [2]:
def create_A_L(inputFileName):
    f = open(inputFileName, 'r')
    lines = f.readlines()
    f.close()
    Adj_mat = []
    for line in lines[0:]:
        l = line.strip('\n\r').split(' ')
        Adj_mat.append(l[0:])
    Adj_mat = np.array(Adj_mat, dtype=np.float32)
    for i in range(157):
        for j in range(i-1,157):
            if i == j:
                Adj_mat[i][j] = 0
            else :
                Adj_mat[i][j] = Adj_mat[i][j]+Adj_mat[j][i]
                Adj_mat[j][i] = Adj_mat[i][j]
    return Adj_mat
def four_ord_ppmi(matrix,node_size):
    def recip_mat(matrix):
        return np.reciprocal(np.sqrt(np.sum(matrix, axis=1)))
    
    dig_value = recip_mat(matrix).tolist()
    dig_value = list(_flatten(dig_value))
    trans_mat = np.mat(np.diag(dig_value))
    trans_A = trans_mat*matrix*trans_mat
    four_ord_mat = (trans_A + trans_A*trans_A + trans_A*trans_A*trans_A + trans_A*trans_A*trans_A*trans_A )/4
    four_ord_mat_value = recip_mat(four_ord_mat)
    dig_value2 = recip_mat(four_ord_mat).tolist()
    dig_value2 = list(_flatten(dig_value2))
    trans_mat2 = np.mat(np.diag(dig_value2))
    trans_B = trans_mat2*four_ord_mat*trans_mat2
    ppmi_mat = np.log(trans_B) - np.log(1/node_size)
    ppmi_mat[np.isnan(ppmi_mat)] = 0.0
    ppmi_mat[np.isinf(ppmi_mat)] = 0.0
    ppmi_mat[np.isneginf(ppmi_mat)] = 0.0
    ppmi_mat[ppmi_mat<0] = 0.0
    return ppmi_mat
#PPMI，特征矩阵
def output_features_mat(trans_mat, feature_mat):
    node_num = feature_mat.shape[0]
    feature_num = feature_mat.shape[1]
    output_feature_mat = np.zeros_like(feature_mat)
    for i in range(node_num):
        trans_weight_sum = 0
        avg_features = np.zeros(shape=(feature_num))
        node_i = np.array(trans_mat[i])[0]
        node_i_index_array = np.nonzero(node_i)[0]
        node_i_index_len = len(node_i_index_array)
        for j in range(node_i_index_len):
            trans_index = node_i_index_array[j]
            trans_weight = node_i[trans_index]
            avg_features += trans_weight * feature_mat[trans_index]
            trans_weight_sum += trans_weight
        avg_features /= trans_weight_sum
        output_feature_mat[i] = avg_features
    return output_feature_mat

def read_feature2(inputFileName):
    f = open(inputFileName, 'r')
    lines = f.readlines()
    f.close()
    features = []
    for line in lines[0:]:
        l = line.strip('\n\r').split(' ')
        features.append(l[0:])
    features = np.array(features, dtype=np.float32)
    return features


In [6]:
input_feature_mat=read_feature2('E:\Program Files\jupyter_code\code\out\out2.txt')*10000
print(input_feature_mat)

[[6.05737991e+01 2.41136990e+01 1.43002996e+01 ... 6.72195969e+01
  4.80415039e+01 2.68676987e+01]
 [1.42530000e+00 5.42000011e-02 3.73400003e-01 ... 0.00000000e+00
  5.33789968e+00 8.39600027e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [7.83899975e+00 4.55690002e+00 1.75489998e+00 ... 1.32402000e+01
  1.06758995e+01 3.77830005e+00]
 [2.85049987e+00 1.89870000e+00 1.12010002e+00 ... 4.07390022e+00
  4.00349998e+00 1.25940001e+00]
 [0.00000000e+00 2.71000005e-02 1.86700001e-01 ... 0.00000000e+00
  0.00000000e+00 4.19800013e-01]]


In [12]:
ccc = np.loadtxt("output.txt")
input_mat = four_ord_ppmi(ccc,157)
output_feature_mat = output_features_mat(input_mat, input_feature_mat)

In [13]:
print(input_mat)
print(output_feature_mat)

[[0.19828122 0.         0.         ... 0.         0.02750338 0.02004384]
 [0.         0.22062004 0.         ... 0.         0.         0.        ]
 [0.         0.         0.22070877 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.21850097 0.03225768 0.        ]
 [0.02750338 0.         0.         ... 0.03225768 0.21361081 0.        ]
 [0.02004384 0.         0.         ... 0.         0.         0.21918053]]
[[8.41549530e+01 2.91609955e+01 1.56001406e+01 ... 1.00548904e+02
  8.41941223e+01 3.83139305e+01]
 [1.42529988e+00 5.42000011e-02 3.73399973e-01 ... 0.00000000e+00
  5.33789968e+00 8.39600027e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [9.85940247e+01 3.17165241e+01 1.79913902e+01 ... 1.06985565e+02
  9.82420197e+01 4.44198494e+01]
 [1.75184235e+02 5.79401131e+01 3.11351566e+01 ... 2.00294952e+02
  1.77323425e+02 7.88019180e+01]
 [1.02511909e+02 3.22511063e+01 1.83086128e+01 ... 1.046

In [14]:
np.savetxt('output_feature.txt', output_feature_mat)

In [ ]:
# 代码行104